# File encryption via Fernet and RSA

In [58]:
from cryptography.fernet import Fernet
import rsa

### Helper functions definition

In [59]:
file_password_secrets = './secrets/pwd.json'

In [60]:
def generate_primary_key():
    """
        Generates the key that is used to encrypt the file
    """
    key = Fernet.generate_key()
    return key

In [61]:
def load_secondary_key():
    """
    Loads the key from the current directory named `key.key` (ciphered primary_key )
    """
    return open("./keys/secondary_key.key", "rb").read()

In [62]:
def load_pem_key(type_key):
    """
    Loads the key .pem
    """
    key_file = './keys/public_key.pem'
    key_type = rsa.PublicKey
    if type_key != 'public':
        key_file = './keys/private_key.pem'
        key_type = rsa.PrivateKey
    
    
    with open(key_file, mode='rb') as key_file:
        key_data = key_file.read()
    
    key = key_type.load_pkcs1(key_data)
    return key

In [63]:
def encrypt_password_bank(filename, primary_key):
    """
    Encrypt file from dir ./secrets
    """
    f = Fernet(primary_key)
    with open(filename, "rb") as file:
    # read all file data
        file_data = file.read()
    encrypted_data = f.encrypt(file_data)

        # write the encrypted file
    with open(filename, "wb") as file:
        file.write(encrypted_data)

In [64]:
def decrypt_password_bank(filename, primary_key):
    """
    Dencrypt file from dir ./secrets
    """
    f = Fernet(primary_key)
    with open(filename, "rb") as file:
        # read the encrypted data
        encrypted_data = file.read()
    # decrypt data
    decrypted_data = f.decrypt(encrypted_data)
    # write the original file
    with open(filename, "wb") as file:
        file.write(decrypted_data)

In [65]:
def save_rsa_pem_keys(public_key, private_key):
    """
    Generates public and private key.pm and saved them into dir ./keys
    """
    public_key_path = './keys/public_key.pem'
    private_key_path = './keys/private_key.pem'

    # write the original file
    with open(public_key_path, "wb") as file:
        file.write(public_key)
    
    with open(private_key_path, "wb") as file:
        file.write(private_key)

In [66]:
def generate_secondary_key(primary_key:bytes):
    """ Ciphers the primary key via RSA """
    (public_key,private_key) = rsa.newkeys(1024)
    save_rsa_pem_keys(
        public_key=public_key.save_pkcs1(), 
        private_key=private_key.save_pkcs1()
    )
    secondary_key = rsa.encrypt(primary_key, public_key)
    with open('./keys/secondary_key.key','wb') as key_file:
        key_file.write(secondary_key)

### Generate primary key

`Primary key` is used to encrypt the file

In [67]:
primary_key = generate_primary_key()

### Encrypt the file using the primary key

In [73]:
encrypt_password_bank(filename=file_password_secrets, primary_key=primary_key)

### Cipher primary key (PK) via RSA

- This creates both pair of keys (public and private) and they both got saved into keys directory
- `Private key .pem `needs to be saved in a secure place, because of `PK is used to decrypt your data`. That's why this key was added within `.gitignore `

In [69]:
generate_secondary_key(primary_key=primary_key)

### Decrypt data

To decrypt data `private key .pem` needs to be added into `keys directory` in order to decrypt the primary key and use it for decrypt the file.

In [70]:
sk = load_secondary_key()
privateKey = load_pem_key('private')
pk = rsa.decrypt(sk, privateKey)

In [72]:
decrypt_password_bank(filename=file_password_secrets, primary_key=pk)